In [217]:
import random
from pulp import *
from collections import deque
import time
from collections import Counter

# Ucitavanje grafa

In [218]:
def load_graph_from_mtx(filename):
    graph = {}
    with open(filename, 'r') as file:
        for i, line in enumerate(file):
            # preskocimo prva tri reda (jer su tu podaci o broju grana, cvorova...)
            if i < 3:
                continue
            
            nodes = line.strip().split()
            node1, node2 = int(nodes[0]), int(nodes[1])

            if node1 not in graph:
                graph[node1] = set()
            if node2 not in graph:
                graph[node2] = set()

            graph[node1].add(node2)
            graph[node2].add(node1)

    return graph


# Generisanje random burning sekvenci

In [219]:

def generate_random_burning_sequence(graph):
    
    S=[] # sekvenca zapaljenih cvoreva
    B= set() # skup zapaljenih cvoreva
    NB= set(graph.keys()) # skup nezapaljenih cvoreva, inicajlno su to svi cvorevi grafa

    current_node = random.choice(list(NB))
    
    S.append(current_node)
    B.add(current_node)
    NB.remove(current_node)
    
    while len(B)<len(graph):
       
        # korak 2: zapaliti sve susjede (koji nisu zapaljeni) od cvorova u skupu zapaljenih, dodajemo ih u B, brisemo iz NB
        
        # Provjera: ako su svi cvorovi u NB zapravo cvorovi koji ce biti zapaljeni u narednom koraku (jer su direktni susjedi
        # zapaljenom cvoru nekom), onda cemo izabrati neki random cvor iz NB cisto kao simbol jos jednog koraka
        
        FB = []
        for b in B:
            neighbors_b1 = graph[b]
            for neighbor in neighbors_b1:
                if neighbor in NB:
                    if neighbor not in FB:
                        FB.append(neighbor)
             
        if set(FB) == NB:
            #print("fb",FB)
            #print("nb", NB)
            random_node = random.choice(list(NB))
            print("S", random_node)
            S.append(random_node)
          
        '''newlyburned = []
        for node in B:
            neighbors = graph[node]  # svi susjedi trenutnog zapaljenog cvora
            for neighbor in neighbors:
                if neighbor in NB:  # ako susjed nije zapaljen
                    newlyburned.append(neighbor)  # dodajemo ga u zapaljene
                    NB.remove(neighbor)  '''
        
        for new in FB:
            B.add(new)
            NB.remove(new)#dodajemo sve susjede u zapaljene
        
        # pravimo listu onih cvorova za koje znamo da ce se u iducem koraku zapaliti. Bolje da izaberemo sljedeci direktni
        # cvor za koji znamo da nece biti zapaljen u narednom koraku
        
        FB2 = []
        for b in B:
            neighbors_b = graph[b]
            for neighbor in neighbors_b:
                if neighbor in NB:
                    if neighbor not in FB2:
                        FB2.append(neighbor)
       
        # korak 3: pronaci iduci cvor koji ce biti direktno zapaljen, dodati ga u S, B i izbrisati iz NB
        
        max_burned = -1
        best_node = None
        candidates_for_best = []
        
        if NB:
            for node in NB:
                if node not in FB2: 
                    candidates_for_best.append(node)
            if candidates_for_best:
                best_node = random.choice(candidates_for_best)
            else:
                best_node = None        
                
            if best_node is not None:
                S.append(best_node)
                B.add(best_node)
                NB.remove(best_node)
            else:
             # Ako nema nijednog najboljeg cvora znaci da su svi preostali cvorovi povezani i mogu biti zapaljeni
             # Dodajemo slucajni cvor iz preostalih nezapaljenih čvorova
                random_node = random.choice(list(NB))
                S.append(random_node)
                B.add(random_node)
                NB.remove(random_node)
           
            
        else:
            break
           
    return S
        
        

def generate_multiple_random_burning_sequences(graph, num_sequences):
    
    sequences = []
    for _ in range(num_sequences):
        sequence = generate_random_burning_sequence(graph)
        if sequence is not None:  
            sequences.append(sequence)
    return sequences


filename = "grafovi/manji/dolphins.mtx"
graph = load_graph_from_mtx(filename)


#max_burn_nodes = len(graph) // 2
num_sequences = 20

burning_sequences = generate_multiple_random_burning_sequences(graph, num_sequences)

# Ispis rezultata
print(f"Generisane burning sekvence:")
for seq in burning_sequences:
    print(seq,"len:", len(seq))


S 12
S 32
S 27
S 54
S 61
S 57
S 32
S 33
S 50
S 36
S 47
S 12
S 13
S 50
S 27
S 62
S 57
Generisane burning sekvence:
[58, 15, 56, 5, 12] len: 5
[17, 49, 23, 61, 32] len: 5
[61, 52, 49, 59, 50, 27] len: 6
[49, 12, 60, 59, 50, 54] len: 6
[16, 18, 50, 54, 61] len: 5
[22, 27, 33, 57, 49] len: 5
[56, 45, 47, 27, 33, 23, 57] len: 7
[22, 8, 57, 61, 32] len: 5
[17, 50, 26, 57, 61, 33] len: 6
[56, 11, 28, 33, 59, 50] len: 6
[8, 3, 52, 50, 36] len: 5
[23, 16, 3, 59, 50, 47] len: 6
[9, 18, 61, 12] len: 4
[7, 12, 43, 47, 59, 13] len: 6
[29, 7, 12, 54, 50] len: 5
[44, 58, 31, 61, 27] len: 5
[28, 3, 50, 12, 5] len: 5
[61, 59, 60, 43, 5, 62] len: 6
[41, 10, 12, 5] len: 4
[51, 26, 61, 42, 57] len: 5


# Merge nasumicno generisane sekvence

In [ ]:


def find_most_frequent_node(NB, FB2, burning_sequences):

    #Funkcija koja pronalazi cvor koji se najcesce pojavljuje u sekvencama, 
    #a koji nije susjed prethodnog zapaljenog cvora i nalazi se u listi sekvenci.
    
    
    valid_nodes = [node for node in NB if node not in FB2]
    
    if not valid_nodes:
        return None 

    node_count = Counter()
    
    for seq in burning_sequences:
        for node in seq:
            if node in valid_nodes:
                node_count[node] += 1
    
    #  cvor koji se najcesce pojavljuje
    most_frequent_node = node_count.most_common(1)
    
    if most_frequent_node:
        return most_frequent_node[0][0]  
    return None

def merge_burning_sequences(graph, burning_sequences):
    S = []  # sekvenca zapaljenih čvorova
    B = set()  # skup zapaljenih čvorova
    NB = set(graph.keys())  # skup nezapaljenih čvorova, inicijalno su to svi čvorovi grafa
    
    # pocetni cvor je odabrani najzastupljeniji cvor iz svih sekvenci
    all_nodes = [node for seq in burning_sequences for node in seq]
    most_common_node = Counter(all_nodes).most_common(1)[0][0]
    
    S.append(most_common_node)
    B.add(most_common_node)
    NB.remove(most_common_node)
    
    while len(B) < len(graph):
        # korak 2: zapaliti sve susjede (koji nisu zapaljeni) od čvorova u skupu zapaljenih, dodajemo ih u B, brišemo iz NB
        FB = []
        for b in B:
            neighbors_b = graph[b]
            for neighbor in neighbors_b:
                if neighbor in NB:
                    if neighbor not in FB:
                        FB.append(neighbor)
        
       
        if set(FB) == NB:
            most_frequent_node = find_most_frequent_node(NB, FB, burning_sequences)
            if most_frequent_node:
                S.append(most_frequent_node)
                B.add(most_frequent_node)
                NB.remove(most_frequent_node)
            else:
                random_node = random.choice(list(NB))
                S.append(random_node)
                B.add(random_node)
                NB.remove(random_node)
        
        for new in FB:
            if new in NB:  
                B.add(new)
                NB.remove(new)  # dodajemo sve susjede u zapaljene
        
        # pravimo listu onih čvorova za koje znamo da će se u idućem koraku zapaliti
        FB2 = []
        for b in B:
            neighbors_b = graph[b]
            for neighbor in neighbors_b:
                if neighbor in NB:
                    if neighbor not in FB2:
                        FB2.append(neighbor)
        
        # korak 3: pronaći idući čvor koji će biti direktno zapaljen, dodati ga u S, B i izbrisati iz NB
        if NB:
            most_frequent_node = find_most_frequent_node(NB, FB2, burning_sequences)
            if most_frequent_node:
                S.append(most_frequent_node)
                B.add(most_frequent_node)
                NB.remove(most_frequent_node)
            else:
                random_node = random.choice(list(NB))
                S.append(random_node)
                B.add(random_node)
                NB.remove(random_node)
        else:
            break  
    
    return S  # vraćamo završnu sekvencu zapaljenih čvorova



merged_sequence = merge_burning_sequences(graph, burning_sequences)
print("Mergeovana sekvenca:", merged_sequence)


Mergeovana sekvenca: [61, 50, 12, 59, 48, 36]


In [221]:
def generate_distance_matrix(G):
    from collections import deque
    n = len(G)
    nodes = list(G.keys())
    index_map = {v: i for i, v in enumerate(nodes)}  
    
    d = [[float('inf')] * n for _ in range(n)]
    
    for start in nodes:
        start_index = index_map[start]
        d[start_index][start_index] = 0
        queue = deque([start])
        while queue:
            current = queue.popleft()
            current_index = index_map[current]
            for neighbor in G[current]:
                neighbor_index = index_map[neighbor]
                if d[start_index][neighbor_index] == float('inf'):
                    d[start_index][neighbor_index] = d[start_index][current_index] + 1
                    queue.append(neighbor)
    
    return d

def ILP_Pulp(L, k, G, burning_sequences=None):
    n = len(G)  
    d = generate_distance_matrix(G) 
    
    m = LpProblem("Graph_burning", LpMinimize)

    # binarne promjenljive
    b = [LpVariable(f"b_{j}", cat='Binary') for j in range(n)]
    x = [[LpVariable(f"x_{r}_{i}", cat='Binary') for i in range(n)] for r in range(k)]

    # funkcija cilja (11)
    m += lpSum(x[r][i] for r in range(k) for i in range(n))

    # ogranicenje (12)
    for i in range(1, k):
        m += lpSum(x[i][j] for j in range(n)) <= lpSum(x[i-1][j] for j in range(n))

    # ogranicenje (13)
    for i in range(k):
        m += lpSum(x[i][j] for j in range(n)) <= 1

    # ogranicenje (14)
    for j in range(n):
        m += b[j] <= lpSum(x[i][k] for i in range(k) for k in range(n) if d[k][j] <= i)

    # ogranicenje (15)
    m += lpSum(b[j] for j in range(n)) == n

    # ogranicenje (17)
    m += lpSum(x[0][j] for j in range(n)) == 1

    # Generisanje spajene sekvence
    best_sequence = merged_sequence 
    
    # provjera svake sekvence iz liste `burning_sequences`
    best_obj_value = float('inf')
    results = []
    
    if best_sequence:
        m_copy = m.deepcopy()

        # Ograničenja za trenutnu sekvencu
        for r, node in enumerate(best_sequence):
            for i in range(n):
                if i == node:
                    m_copy += x[r][i] == 1
                else:
                    m_copy += x[r][i] == 0
        
        m_copy.solve()
        status = LpStatus[m_copy.status]
        obj_value = value(m_copy.objective)

        results.append({
            "sequence": best_sequence,
            "status": status,
            "objective_value": obj_value
        })
        
        # Ažuriraj najbolju sekvencu
        if status == "Optimal" and obj_value < best_obj_value:
            best_obj_value = obj_value
    
    return results, best_sequence, best_obj_value



startTime = time.time()

k = 10
results, best_sequence, best_obj_value = ILP_Pulp(LpMinimize, k, graph, burning_sequences=burning_sequences)

# Ispis najbolje sekvence
print("\nBest Sequence:")
print(f"Sequence: {best_sequence}")
print(f"Objective Value: {best_obj_value}")

print("--- %s sekundi ---" % (time.time() - startTime))


Best Sequence:
Sequence: [61, 50, 12, 59, 48, 36]
Objective Value: 7.0
--- 0.2582676410675049 sekundi ---
